In [1]:
from MTCFeatures import MTCFeatureLoader
from fractions import Fraction
import pandas as pd
import os
from collections import defaultdict
epsilon = 0.0001
import pickle
from itertools import chain

In [2]:
def findFirstSequencedNote(seq):
    for ix, val in enumerate(seq['features']['nextisrest']):
        if ix==0: continue #skip first
        if not val: #there is no rest following
            return ix

# Grouper

Read the melody sequences

In [ ]:
seqs_mtc = MTCFeatureLoader('ismir2020_seqs_mtc.jsonl.gz').sequences()
seqs_mtc = list(seqs_mtc)

In [ ]:
seqs_chor = MTCFeatureLoader('ismir2020_seqs_chor.jsonl.gz').sequences()
seqs_chor = list(seqs_chor)

In [ ]:
seqs_essen = MTCFeatureLoader('ismir2020_seqs_essen.jsonl.gz').sequences()
seqs_essen = list(seqs_essen)

Function to generate notelist as input for melisma

In [ ]:
def seq2notelist(seq):
    duration_frac = seq['features']['duration_frac']
    restduration_frac = seq['features']['restduration_frac']
    onsettick = seq['features']['onsettick']
    midipitch = seq['features']['midipitch']
    #find out length of onset tick
    #find index of first note without rest following
    ix = findFirstSequencedNote(seq)
    tick_duration = Fraction(duration_frac[ix]) / ( Fraction(onsettick[ix+1]) - Fraction(onsettick[ix]) )
    onset = 0
    notes = []
    for ix, val in enumerate(duration_frac):
        dur_ticks = int(Fraction(val) / tick_duration)
        if restduration_frac[ix] is not None:
            rest_ticks = int(Fraction(restduration_frac[ix]) / tick_duration)
            #print(ix, onset, "rest")
        else:
            rest_ticks = 0
        offset = onset + dur_ticks
        notes.append( (onset, offset, midipitch[ix]) )
        onset = onset + dur_ticks + rest_ticks
    #check whether computed onsets are same as provided onsets
    for ix, val in enumerate(notes):
        if notes[ix][0] != onsettick[ix]:
            print(f"{seq['id']}: Difference in onset at position {ix}, onsettick: {onsettick[ix]}")
            break
    #multiply by 100
    notes = [ (onset*100, offset*100, midipitch) for onset, offset, midipitch in notes ]
    return notes
                  

In [ ]:
def seq2notelistBeatduration(seq):
    # use beat = 1 second (=1000ms)
    beatfraction = seq['features']['beatfraction']
    duration_frac = seq['features']['duration_frac']
    restduration_frac = seq['features']['restduration_frac']
    midipitch = seq['features']['midipitch']
    beatlength = [ Fraction(dur) / Fraction(bf) for dur, bf in zip(duration_frac, beatfraction)]
    restdurationbeat_frac = [ Fraction(rd) / bl if rd is not None else None for rd, bl in zip(restduration_frac, beatlength) ]
    notes = []
    onset = 0
    for ix, bf in enumerate(beatfraction):
        notedur = Fraction(1000) * ( Fraction(bf) )
        if restdurationbeat_frac[ix] is not None:
            totaldur = Fraction(1000) * ( Fraction(bf) + restdurationbeat_frac[ix] )
        else:
            totaldur = notedur
        notes.append( (onset, onset+int(notedur), midipitch[ix] ) )
        onset += int(totaldur)
    return notes

In [ ]:
def writeNoteList(notelist, filename):
    with open(filename, 'w') as f:
        for ix, note in enumerate(notelist):
            f.write(f'Note {note[0]} {note[1]} {note[2]}\n')

Generate the notelist files

In [ ]:
if True:
    for seq in seqs_mtc:
        writeNoteList( seq2notelistBeatduration(seq), f"ismir2020_melisma/notefiles/mtcfsinst/{seq['id']}.notes" )

    for seq in seqs_essen:
        writeNoteList( seq2notelistBeatduration(seq), f"ismir2020_melisma/notefiles/essen/{seq['id']}.notes" )

    for seq in seqs_chor:
        writeNoteList( seq2notelistBeatduration(seq), f"ismir2020_melisma/notefiles/chor/{seq['id']}.notes" )

in command shell: invoke melisma meter

Insert phrase ends in .nb files

In [ ]:
def insertPhraseEnd(path, seq):
    songid = seq['id']
    prhaseend = seq['features']['phrase_end']
    with open(os.path.join(path,songid+'.nb'), 'r') as f:
        lines = f.read().split('\n')
    with open(os.path.join(path,songid+'.nb'), 'w') as f:
        ix = 0
        for line in lines:
            f.write(line)
            f.write('\n')
            if line[:4] == 'Note':
                if prhaseend[ix]:
                    f.write("|\n")
                ix += 1

In [ ]:
path = 'ismir2020_melisma/nbfiles/mtcfsinst'
for seq in seqs_mtc:
    try:
        insertPhraseEnd(path, seq)
    except FileNotFoundError as e:
        print(e)

In [ ]:
path = 'ismir2020_melisma/nbfiles/essen'
for seq in seqs_essen:
    try:
        insertPhraseEnd(path, seq)
    except FileNotFoundError as e:
        print(e)

In [ ]:
path = 'ismir2020_melisma/nbfiles/chor'
for seq in seqs_chor:
    try:
        insertPhraseEnd(path, seq)
    except FileNotFoundError as e:
        print(e)

Now read groupers phrase boundaries as feature

In [ ]:
for seq in seqs_mtc:
    with open(f"ismir2020_melisma/boundaries/mtcfsinst/{seq['id']}.bd", 'r') as f:
        lines = f.read().split('\n')
    grouper_boundaries = []
    for pair in zip(lines, lines[1:]):
        if pair[0][:4] == 'Note':
            if pair[1][:6] == 'Phrase':
                grouper_boundaries.append(True)
            else:
                grouper_boundaries.append(False)
    #check length
    if len(grouper_boundaries) != len(seq['features']['scaledegree']):
        print(f'{seq[id]}: unequal lengths')
    seq['features'] = {}
    seq['features']['grouper'] = grouper_boundaries

MTCFeatureLoader.writeJSON('ismir2020_melisma/mtcfsinst_grouper.jsonl.gz', seqs_mtc)

In [ ]:
for seq in seqs_essen:
    with open(f"ismir2020_melisma/boundaries/essen/{seq['id']}.bd", 'r') as f:
        lines = f.read().split('\n')
    grouper_boundaries = []
    for pair in zip(lines, lines[1:]):
        if pair[0][:4] == 'Note':
            if pair[1][:6] == 'Phrase':
                grouper_boundaries.append(True)
            else:
                grouper_boundaries.append(False)
    #check length
    if len(grouper_boundaries) != len(seq['features']['scaledegree']):
        print(f'{seq[id]}: unequal lengths')
    seq['features'] = {}
    seq['features']['grouper'] = grouper_boundaries

MTCFeatureLoader.writeJSON('ismir2020_melisma/essen_grouper.jsonl.gz', seqs_essen)

In [ ]:
for seq in seqs_chor:
    with open(f"ismir2020_melisma/boundaries/chor/{seq['id']}.bd", 'r') as f:
        lines = f.read().split('\n')
    grouper_boundaries = []
    for pair in zip(lines, lines[1:]):
        if pair[0][:4] == 'Note':
            if pair[1][:6] == 'Phrase':
                grouper_boundaries.append(True)
            else:
                grouper_boundaries.append(False)
    #check length
    if len(grouper_boundaries) != len(seq['features']['scaledegree']):
        print(f'{seq[id]}: unequal lengths')
    seq['features'] = {}
    seq['features']['grouper'] = grouper_boundaries

MTCFeatureLoader.writeJSON('ismir2020_melisma/chor_grouper.jsonl.gz', seqs_chor)

# IDyOM

Add IDyOM information content from IDyOM to the sequences.

In [ ]:
# reload the sequences and make dict
seqs_mtc = MTCFeatureLoader('ismir2020_seqs_mtc.jsonl.gz').sequences()
seqs_mtc = list(seqs_mtc)

#have a dict
seqs_mtc_dict = { seq['id'] : seq for seq in seqs_mtc}

In [ ]:
seqs_chor = MTCFeatureLoader('ismir2020_seqs_chor.jsonl.gz').sequences()
seqs_chor = list(seqs_chor)

#have a dict
seqs_chor_dict = { seq['id'] : seq for seq in seqs_chor}

In [ ]:
seqs_essen = MTCFeatureLoader('ismir2020_seqs_essen.jsonl.gz').sequences()
seqs_essen = list(seqs_essen)

#have a dict
seqs_essen_dict = { seq['id'] : seq for seq in seqs_essen}    

Select some columns from IDyOM's output

In [ ]:
def reduceIdyomDF(df):
    #only retain columns with ID, phrase and information.content
    df = df.loc[:,['dataset.id','melody.id','note.id','melody.name','phrase','information.content']]
    df.rename(
        columns={
            'dataset.id': 'dataset_id',
            'melody.id': 'melody_id',
            'note.id': 'note_id',
            'melody.name': 'melody_name',
            'information.content': 'information_content'
        },
        inplace=True
    )

    #make meaningful index
    df['id'] = df.apply(
        lambda row: '-'.join(
            [
                str(row.melody_name),
                str(row.note_id)
            ]
        ),
        axis = 1
    )
    df.set_index(['id'])
    return df

In [ ]:
idyom_out_mtc = pd.read_csv(
    '/Users/krane108/Documents/Eigenwerk/Projects/ScaleDegrees/IDyOM/3-cpitch_bioi_deltast-cpitch_bioi_deltast-nil-nil-melody-nil-10-both+-nil-t-nil-c-nil-t-t-x-3.csv',
    sep = ' '
)
idyom_out_essen = pd.read_csv(
    '/Users/krane108/Documents/Eigenwerk/Projects/ScaleDegrees/IDyOM/4-cpitch_bioi_deltast-cpitch_bioi_deltast-nil-nil-melody-nil-10-both+-nil-t-nil-c-nil-t-t-x-3.csv',
    sep = ' '
)
idyom_out_chor  = pd.read_csv(
    '/Users/krane108/Documents/Eigenwerk/Projects/ScaleDegrees/IDyOM/2-cpitch_bioi_deltast-cpitch_bioi_deltast-nil-nil-melody-nil-10-both+-nil-t-nil-c-nil-t-t-x-3.csv',
    sep = ' '
)

idyom_out_mtc = reduceIdyomDF(idyom_out_mtc)
idyom_out_chor = reduceIdyomDF(idyom_out_chor)
idyom_out_essen = reduceIdyomDF(idyom_out_essen)

Write seqences to disk.

In [ ]:
icfeats_mtc = defaultdict(list)

for row in idyom_out_mtc.values:
    icfeats_mtc[row[3]].append(row[5])
    
for songid in icfeats_mtc.keys():
    #check lengths
    if len(seqs_mtc_dict[songid]['features']['scaledegree']) != len(icfeats_mtc[songid]):
        print("Unequal lengths: " + songid)
    seqs_mtc_dict[songid]['features'] = {}
    seqs_mtc_dict[songid]['features']['informationcontent'] = icfeats_mtc[songid]
    
MTCFeatureLoader.writeJSON('ismir2020_melisma_IDyOM_sel/mtcfsinst_vocal_meter_after1850_1pertf_informationcontent.jsonl.gz', seqs_mtc)

In [ ]:
icfeats_essen = defaultdict(list)

for row in idyom_out_essen.values:
    icfeats_essen[row[3]].append(row[5])

for songid in icfeats_essen.keys():
    #check lengths
    if len(essen_seqs_dict[songid]['features']['scaledegree']) != len(icfeats_essen[songid]):
        print("Unequal lengths: " + songid)
    essen_seqs_dict[songid]['features'] = {}
    essen_seqs_dict[songid]['features']['informationcontent'] = icfeats_essen[songid]

MTCFeatureLoader.writeJSON('ismir2020_melisma_IDyOM_sel/essen_erk_meter_informationcontent.jsonl.gz', essen_seqs)   

In [ ]:
icfeats_chor = defaultdict(list)

for row in idyom_out_chor.values:
    icfeats_chor[row[3]].append(row[5])

for songid in icfeats_chor.keys():
    #check lengths
    if len(chor_seqs_dict[songid]['features']['scaledegree']) != len(icfeats_chor[songid]):
        print("Unequal lengths: " + songid)
    chor_seqs_dict[songid]['features'] = {}
    chor_seqs_dict[songid]['features']['informationcontent'] = icfeats_chor[songid]

MTCFeatureLoader.writeJSON('ismir2020_melisma_IDyOM_sel/chor_meter_informationcontent.jsonl.gz', chor_seqs)

# Baselines

Compute the rest baseline

In [3]:
# reload the sequences

seqs_mtc = MTCFeatureLoader('ismir2020_seqs_mtc.jsonl.gz').sequences()
seqs_mtc = list(seqs_mtc)

#selection for ismir2020
with open('ismir2020_songids_mtc.txt', 'r') as f:
    songids_mtc = [line.rstrip() for line in f.readlines()]
seqs_mtc = [seq for seq in seqs_mtc if seq['id'] in songids_mtc]


In [4]:
seqs_essen = MTCFeatureLoader('ismir2020_seqs_essen.jsonl.gz').sequences()
seqs_essen = list(seqs_essen)

#selection for ismir2020
with open('ismir2020_songids_essen.txt', 'r') as f:
    songids_essen = [line.rstrip() for line in f.readlines()]
seqs_essen = [seq for seq in seqs_essen if seq['id'] in songids_essen]    


In [5]:
seqs_chor = MTCFeatureLoader('ismir2020_seqs_chor.jsonl.gz').sequences()
seqs_chor = list(seqs_chor)

#selection for ismir2020
with open('ismir2020_songids_chor.txt', 'r') as f:
    songids_chor = [line.rstrip() for line in f.readlines()]
seqs_chor = [seq for seq in seqs_chor if seq['id'] in songids_chor]


In [6]:
len(seqs_mtc),len(seqs_essen),len(seqs_chor)

(1323, 1632, 370)

In [7]:
def evaluate(TP, FP, FN):
    precision = TP / (TP+FP)
    recall = TP / (TP+FN)
    F1 = 2*precision*recall / (precision+recall)
    print(f"Pr: {precision}")
    print(f"Rc: {recall}")
    print(f"F1: {F1}")

Count True Positives, False Positives, and False Negatives

In [8]:
def restPredictions(corpus):
    TP,FP,FN = 0,0,0
    for seq in corpus:
        feats = list(zip(seq['features']['nextisrest'],seq['features']['phrase_end']))
        for rest,phraseend in feats[:-1]:
            if rest and phraseend: TP += 1
            if rest and not phraseend: FP += 1
            if not rest and phraseend: FN +=1
    return TP, FP, FN

In [9]:
evaluate(*restPredictions(seqs_mtc))

Pr: 0.9213483146067416
Rc: 0.2557051736357194
F1: 0.4003106623765672


In [10]:
evaluate(*restPredictions(seqs_essen))

Pr: 0.9535529972211195
Rc: 0.31182656108009865
F1: 0.4699667384073567


In [11]:
evaluate(*restPredictions(seqs_chor))

Pr: 0.9942857142857143
Rc: 0.09124278972207656
F1: 0.16714697406340057


Compute always baseline

In [12]:
def always(pos, neg):
    TP = pos
    FP = neg
    FN = 0
    evaluate(TP, FP, FN)


In [13]:
#MTC
always(7054, 63856)

Pr: 0.0994782118177972
Rc: 1.0
F1: 0.18095531270842952


In [14]:
#ESSEN
always(7703, 62490)

Pr: 0.10974028749305487
Rc: 1.0
F1: 0.1977765225428777


In [15]:
#CHOR
always(1907, 15455)

Pr: 0.10983757631609262
Rc: 1.0
F1: 0.19793450620167108


# LBDM

Compute LBDM boundaries

In [16]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [17]:
# reload the sequences and make dict
with open('ismir2020_songids_mtc.txt', 'r') as f:
    songids_mtc = [line.rstrip() for line in f.readlines()]

vocal_seqs = MTCFeatureLoader('ismir2020_seqs_mtc.jsonl.gz').sequences()
vocal_seqs = [seq for seq in vocal_seqs if seq['id'] in songids_mtc]

#have a dict
vocal_seqs_dict = { seq['id'] : seq for seq in vocal_seqs}

In [18]:
with open('ismir2020_songids_essen.txt', 'r') as f:
    songids_essen = [line.rstrip() for line in f.readlines()]

essen_seqs = MTCFeatureLoader('ismir2020_seqs_essen.jsonl.gz').sequences()
essen_seqs = [seq for seq in essen_seqs if seq['id'] in songids_essen]

#have a dict
essen_seqs_dict = { seq['id'] : seq for seq in essen_seqs}   

In [19]:
with open('ismir2020_songids_chor.txt', 'r') as f:
    songids_chor = [line.rstrip() for line in f.readlines()]

chor_seqs = MTCFeatureLoader('ismir2020_seqs_chor.jsonl.gz').sequences()
chor_seqs = [seq for seq in chor_seqs if seq['id'] in songids_chor]

#have a dict
chor_seqs_dict = { seq['id'] : seq for seq in chor_seqs}

In [20]:
def computeLBDMBoundaries(corpus, percentage):
    #first collect all values
    allLBDM = [seq['features']['lbdm_boundarystrength'] for seq in corpus]
    allLBDM = list(chain.from_iterable(allLBDM))
    allLBDM = [val if val is not None else 0 for val in allLBDM]
    allLBDM = sorted(allLBDM, reverse=True)
    ix = int(len(allLBDM) * percentage)
    threshold = allLBDM[ix]
    #collect boundaries and groundtruth
    y = []
    y_pred = []
    for seq in corpus:
        for bd in seq['features']['lbdm_boundarystrength'][:-1]:
            if bd is not None:
                y_pred.append(1 if bd>=threshold else 0)
            else:
                y_pred.append(0)
        for bd in seq['features']['endOfPhrase'][:-1]:
            if bd:
                y.append(1)
            else:
                y.append(0)
    return y, y_pred

Search optimal threshold

In [21]:
def search(corpus):
    res = []
    for percentage in range(1,100):
        if percentage%10==0: print(percentage)
        p = float(percentage) / 100.0
        y, y_pred = computeLBDMBoundaries(corpus, p)
        f1 = f1_score(y,y_pred)
        res.append((p, f1))
    res = sorted(res, key = lambda x: x[1], reverse=True)
    print (res[0])
    y, y_pred = computeLBDMBoundaries(corpus, res[0][0])
    print(classification_report(y, y_pred))

In [22]:
search(vocal_seqs)

10
20
30
40
50
60
70
80
90
(0.08, 0.5488272267361644)
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     66501
           1       0.60      0.51      0.55      7055

    accuracy                           0.92     73556
   macro avg       0.77      0.74      0.75     73556
weighted avg       0.91      0.92      0.92     73556



In [23]:
search(essen_seqs)

10
20
30
40
50
60
70
80
90
(0.08, 0.5262698931763681)
              precision    recall  f1-score   support

           0       0.94      0.96      0.95     65754
           1       0.60      0.47      0.53      7703

    accuracy                           0.91     73457
   macro avg       0.77      0.72      0.74     73457
weighted avg       0.90      0.91      0.91     73457



In [24]:
search(chor_seqs)

10
20
30
40
50
60
70
80
90
(0.09, 0.4504201680672269)
              precision    recall  f1-score   support

           0       0.93      0.95      0.94     16195
           1       0.48      0.42      0.45      1907

    accuracy                           0.89     18102
   macro avg       0.71      0.68      0.70     18102
weighted avg       0.89      0.89      0.89     18102

